In [1]:
%reload_kedro

2023-10-10 00:10:57,836 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: C:\Users\Dushku\PycharmProjects\monash-datahack-2023
2023-10-10 00:10:57,958 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-10-10 00:10:57,978 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-telemetry-0.2.5
2023-10-10 00:10:58,037 - kedro.extras.extensions.ipython - INFO - ** Kedro project Monash datahack 2023
2023-10-10 00:10:58,038 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


C:\Users\Dushku\PycharmProjects\monash-datahack-2023\venv\lib\site-packages\kedro\framework\context\context.py:344: UserWarning: Credentials not found in your Kedro project config.
No files found in ['C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\base', 'C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [2]:
catalog.list()

['raw_data',
 'preprocessed_data_with_type_classification',
 'preprocessed_data_with_total_classification',
 'preprocessed_data_not_recycled',
 'preprocessed_data_08_12_13',
 'preprocessed_data_not_08_12_13',
 'preprocessed_data_with_mining',
 'preprocessed_data_without_mining',
 'preprocessed_data_19_21',
 'preprocessed_data_not_19_21',
 'preprocessed_data_without_australia',
 'encoded_data_without_australia',
 'preprocessed_data_with_australia',
 'encoded_data_with_australia',
 'encoded_data_19_21',
 'encoded_data_not_19_21',
 'encoded_data_with_type_classification',
 'encoded_data_with_total_classification',
 'encoded_data_without_zero_values',
 'encoded_data_not_recycled',
 'encoded_data_08_12_13',
 'encoded_data_not_08_12_13',
 'encoded_data_with_mining',
 'encoded_data_without_mining',
 'companies',
 'reviews',
 'shuttles',
 'data_processing.preprocessed_companies',
 'data_processing.preprocessed_shuttles',
 'model_input_table',
 'data_science.active_modelling_pipeline.regressor'

In [3]:
df = catalog.load('preprocessed_data_with_type_classification')

2023-10-10 00:10:58,068 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_with_type_classification` (CSVDataSet)...


In [4]:
df.head()

,Jurisdiction,Category,Type,Management,Fate,y,ds,Sub_stream_name,Sub_stream_index
0,ACT,Biosolids,Biosolids,Other disposal,Disposal,20186,2021-12-31,C&I core,3
1,ACT,Biosolids,Biosolids,Recycling,Recycling,0,2021-12-31,C&I core,3
2,ACT,Biosolids,Biosolids,Other disposal,Disposal,20186,2021-12-31,Total,9
3,ACT,Biosolids,Biosolids,Recycling,Recycling,0,2021-12-31,Total,9
4,ACT,Building and demolition materials,Asphalt,Recycling,Recycling,0,2021-12-31,C&D,2


In [5]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Management', 'Fate', 'y', 'ds',
       'Sub_stream_name', 'Sub_stream_index'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54227 entries, 0 to 54226
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Jurisdiction      54227 non-null  object
 1   Category          54227 non-null  object
 2   Type              54227 non-null  object
 3   Management        54227 non-null  object
 4   Fate              54227 non-null  object
 5   y                 54227 non-null  int64 
 6   ds                54227 non-null  object
 7   Sub_stream_name   54227 non-null  object
 8   Sub_stream_index  54227 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 3.7+ MB


In [7]:
df['y'] = df['y'].astype(int)

In [8]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [9]:
def encode(df):
    df['y'] = df['y'].astype(int)
    label_encoders = {}

    for col in df.columns:
        if df[col].dtype == 'object':  # Check if the column contains categorical data
            label_encoders[col] = LabelEncoder()
            df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])

    df['y'] = LabelEncoder().fit_transform(df['y'])
    
    return df

In [10]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Management', 'Fate', 'y', 'ds',
       'Sub_stream_name', 'Sub_stream_index', 'Jurisdiction_encoded',
       'Category_encoded', 'Type_encoded', 'Management_encoded',
       'Fate_encoded', 'ds_encoded', 'Sub_stream_name_encoded'],
      dtype='object')

In [11]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 
         'Sub_stream_name', 
         'Sub_stream_index', 
         'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54227 entries, 0 to 54226
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   y                     54227 non-null  int64 
 1   Jurisdiction_encoded  54227 non-null  int32 
 2   Sub_stream_name       54227 non-null  object
 3   Sub_stream_index      54227 non-null  int64 
 4   Category_encoded      54227 non-null  int32 
 5   Type_encoded          54227 non-null  int32 
 6   Management_encoded    54227 non-null  int32 
 7   Fate_encoded          54227 non-null  int32 
 8   ds_encoded            54227 non-null  int32 
dtypes: int32(6), int64(2), object(1)
memory usage: 2.5+ MB


In [13]:
catalog.save("encoded_data_with_type_classification", df)

2023-10-10 00:10:58,566 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_with_type_classification` (CSVDataSet)...


In [14]:
df = catalog.load('preprocessed_data_with_total_classification')

2023-10-10 00:10:58,627 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_with_total_classification` (CSVDataSet)...


In [15]:
df['y'] = df['y'].astype(int)

In [16]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [17]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Total type', 'Management', 'Fate',
       'y', 'ds', 'Sub-stream', 'Jurisdiction_encoded', 'Category_encoded',
       'Type_encoded', 'Total type_encoded', 'Management_encoded',
       'Fate_encoded', 'ds_encoded', 'Sub-stream_encoded'],
      dtype='object')

In [18]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Total type', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [19]:
catalog.save("encoded_data_with_total_classification", df)

2023-10-10 00:10:58,706 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_with_total_classification` (CSVDataSet)...


In [20]:
df = catalog.load('preprocessed_data_not_recycled')

2023-10-10 00:10:58,721 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_not_recycled` (CSVDataSet)...


In [21]:
df['y'] = df['y'].astype(int)

In [22]:
# List of columns to exclude
exclude_columns = ['y', 'ds_encoded', ]

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through columns and get unique values
for column in df.columns:
    if column not in exclude_columns:
        unique_values_dict[column] = df[column].unique()

# Print unique values for each column
for column, unique_values in unique_values_dict.items():
    print(f"Unique values for {column}:")
    print(unique_values)
    print()

Unique values for Jurisdiction:
['ACT' 'NSW' 'NT' 'Qld' 'SA' 'Tas' 'Vic' 'WA']

Unique values for Category:
['Biosolids' 'Building and demolition materials' 'Glass'
 'Hazardous wastes' 'Metals' 'Mining' 'Organics' 'Paper & cardboard'
 'Plastics' 'Textiles, leather & rubber (excl. tyres)' 'Total'
 'Unclassified materials' 'Ash']

Unique values for Type:
['Biosolids' nan
 'Soil, sand and rock not contaminated above any threshold requiring classification as contaminated soils (N120)'
 'Acids (B)' 'Alkalis (C)' 'Asbestos (N220)'
 'Clinical and pharmaceutical (R)' 'Contaminated soils (N120)'
 'Food-derived hazardous wastes (K100, K110)' 'Inorganic chemicals (D)'
 'Oils (J)' 'Organic chemicals (M)' 'Organic solvents (G)' 'Other'
 'Other hazardous organic wastes (K140, K190)'
 'Other miscellaneous (other T)' 'Other soil/sludges (other N)'
 'Paints, resins, inks, organic sludges (F)' 'Pesticides (H)'
 'Plating and heat treatment (A)' 'Reactive chemicals (E)' 'Tyres (T140)'
 'Food organics' 'Ga

In [23]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [24]:
import joblib
import os

# Define the folder name within your project
folder_name = 'monash-datahack-2023/app/encoders'

# Create the full path to the folder
folder_path = os.path.join(os.getcwd(), folder_name)

# Ensure that the folder exists; if not, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the LabelEncoder objects to separate files
for col, encoder in label_encoders.items():
    encoder_filename = f'{col}_label_encoder.pkl'
    encoder_path = os.path.join(folder_path, encoder_filename)
    joblib.dump(encoder, encoder_path)

In [25]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Classification', 'Total type',
       'Stream', 'Management', 'Fate', 'y', 'Core or non-core', 'Sub-stream',
       'Cat. order', 'Cat. no.', 'Type order', 'Type no.', 'Sub_stream_name',
       'Sub_stream_index', 'Start date', 'ds', 'Jurisdiction_encoded',
       'Category_encoded', 'Type_encoded', 'Classification_encoded',
       'Total type_encoded', 'Stream_encoded', 'Management_encoded',
       'Fate_encoded', 'Core or non-core_encoded', 'Sub-stream_encoded',
       'Cat. order_encoded', 'Cat. no._encoded', 'Type order_encoded',
       'Type no._encoded', 'Sub_stream_name_encoded', 'Start date_encoded',
       'ds_encoded'],
      dtype='object')

In [26]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub_stream_name_encoded', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [27]:
df.head()

,y,Jurisdiction_encoded,Sub_stream_name_encoded,Category_encoded,Type_encoded,Management_encoded,Fate_encoded,ds_encoded
0,4737,0,3,1,6,2,0,14
1,4737,0,5,1,6,2,0,14
2,4737,0,3,1,42,2,0,14
3,4737,0,5,1,42,2,0,14
4,4737,0,3,1,42,2,0,14


In [28]:
catalog.save("encoded_data_not_recycled", df)

2023-10-10 00:10:58,973 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_not_recycled` (CSVDataSet)...


In [29]:
preprocessed_data_names = ['preprocessed_data_08_12_13', 
                           'preprocessed_data_not_08_12_13', 
                           'preprocessed_data_with_mining', 
                           'preprocessed_data_without_mining',
                           'preprocessed_data_19_21',
                           'preprocessed_data_not_19_21',
                           'preprocessed_data_without_australia',
                          'preprocessed_data_with_australia']
encoded_data_names = ['encoded_data_08_12_13', 
                      'encoded_data_not_08_12_13', 
                      'encoded_data_with_mining', 
                      'encoded_data_without_mining',
                      'encoded_data_19_21',
                      'encoded_data_not_19_21',
                      'encoded_data_without_australia',
                      'encoded_data_with_australia',
                     ]

In [30]:
for i in range(len(preprocessed_data_names)):
    df = encode(catalog.load(preprocessed_data_names[i]))
    df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]
    catalog.save(encoded_data_names[i],df)

2023-10-10 00:10:59,049 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_08_12_13` (CSVDataSet)...


KeyError: "['Sub-stream'] not in index"